In [1]:
import os
from bs4 import BeautifulSoup
from natasha import NamesExtractor  
import json
extractor = NamesExtractor()
import re
#регулярка для имён
fio = re.compile("[А-ЯЁ][а-яё]+ [А-ЯЁ]\.[А-ЯЁ]\.")
fio_cifer = re.compile("[А-ЯЁ][а-яё]+ ФИО[0-9]{1,3}")
fio_short = re.compile("ФИО[0-9]{1,3}")
import pandas as pd

### Собираем готовые метаданные

Функция, которая возвращает эксплицитно указанные метаданные и очищенный от тегов текст, а также создаёт словарь, который включает другие метаданные (прокурора, адвоката, секретаря, обвиняемого):

In [2]:
def parse_xml(path):   # парсим xml
    meta_data = {'number': path.split("/")[-1], 'court': 'not_found', 'judge': 'not_found', 'prosecutor': 'not_found', 'advocate': 'not_found', 'secretary': 'not_found', 'accused': 'not_found', 'result': 'not_found', 'category': 'not_found'}
    with open(path, 'r', encoding='utf-8')as f: 
                
        soup = BeautifulSoup(f.read(), 'html.parser')
        meta_data['court'] = soup.court.string     # добавляем в метаданные то, что имеется в разметке
        meta_data['judge'] = soup.judge.string
        meta_data['result'] = soup.result.string
        meta_data['category'] = soup.category.string
        
        html = []
        for line in soup.body:
            line = line.replace('[', '')
            line = line.replace(']', '') 
            html.append(line)
        true_html = ' '.join(html)
                
        html_soup = BeautifulSoup(true_html, 'html.parser') 
    return meta_data, html_soup.get_text()

Функция на всякий случай -- если потребуется достать что-то с разметкой:

In [3]:
def open_with_xml(path):
    text = ""
    with open(path, 'r', encoding='utf-8')as f: 
        for line in f.readlines():
            text += line
    return text

Функция, которая разбивает текст на части (вступление, основная часть, заключение):

In [4]:
def get_parts(text):    # делим на части
    lines = [line for line in text.split('\n')]
    beg, end = 0, 0 
    for num, line in enumerate(lines):
        if 'установил' in line.lower() or 'у с т а н о в и л' in line.lower(): 
            beg = num + 1
        if 'приговорил' in line.lower() or 'п р и г о в о р и л' in line.lower() or 'определил' in line.lower() or 'о п р е д е л и л' in line.lower()  or 'решил' in line.lower() or 'р е ш и л' in line.lower() or "постановил" in line.lower() or "п о с т а н о в и л" in line.lower(): 
            end = num + 1 
        if beg and end:
            beginning = [stroka.strip() for stroka in lines[:beg]]
            main_part = [stroka.strip() for stroka in lines[beg:end]]
            ending = [stroka.strip() for stroka in lines[end:]]
            return [' '.join(main_part), '\n'.join(beginning), '\n'.join(ending)]
        
        # для извлечения метаданных берём get_parts(text)[1:]
        # для всего остального - get_parts(text)[0]

Функция, которая достаёт из строчки русское имя в формате И.И. Иванов:

In [5]:
def get_names(line):
    name = ""
    matches = extractor(line)
    for match in matches:
        fact = match.fact.as_json 
        name = fact["first"]+"."+fact["middle"]+". "+fact["last"].capitalize()
    return name

Функция, которая:
- разбивает текст на три основные части с помощью get_parts
- разбивает вступление либо по запятой (по умолчанию), либо по переносу строки (если запятая не отделяет одни сущности от других)

In [6]:
def splitting_text(path):
    lines = []
    trouble_counter = 0
    text = get_parts(parse_xml(path)[1])[1]
    text = text.split("с участием")
    for part in text:
        lines += [line for line in part.split("\n") if line.strip()]
    for line in lines:
        if len(fio.findall(line)) > 1:
            trouble_counter += 1
    if trouble_counter != 0:
        for part in text:
            lines = [line for line in part.split(",") if line.strip()]
    return lines       

Функция, которая достаёт строки рядом с нужными нам ключевыми словами, в которых содержатся ФИО:

In [8]:
def meta_lines(path):
    lines = splitting_text (path) 
    add_meta = []
    for i, line in enumerate(lines):  
        if "защитник" in line.lower() or "адвокат" in line.lower(): 
            if fio.findall(line):
                add_meta.append(("advocate", line)) 
            else:
                add_meta.append(("advocate", line+lines[i+1])) 
        if "прокурор" in line.lower() or "обвинител" in line.lower():  
            if fio.findall(line):
                add_meta.append(("prosecutor", line)) 
            else:
                add_meta.append(("prosecutor", line+lines[i+1]))  
        if "секретар" in line.lower():
            if fio.findall(line):
                add_meta.append(("secretary", line)) 
            else:
                add_meta.append(("secretary", line+lines[i+1])) 
        if "подсудим" in line.lower() or "в отношении" in line.lower():
            if fio_cifer.findall(line):
                add_meta.append(("accused", line)) 
            elif fio_short.findall(line):
                add_meta.append(("accused", line)) 
            elif fio.findall(line): 
                add_meta.append(("accused", line)) 
            else:
                if i+1 < len(lines):
                    add_meta.append(("accused", line+lines[i+1])) 
    return add_meta

Функция, которая достаёт из полученных выше строк ФИО формата: ("роль в суде", "ФИО"):

In [9]:
def get_meta_names(meta_lines):
    meta_names = []
    for line in meta_lines:         
        matches = extractor(line[1]) 
        if line[0] == "accused":
            if fio_cifer.findall(line[1]):
                meta_names.append((line[0], fio_cifer.findall(line[1])[0]))
            elif fio_short.findall(line[1]):
                meta_names.append((line[0], fio_short.findall(line[1])[0]))
            elif matches:
                try:
                    for match in matches:
                        fact = match.fact.as_json 
                        meta_names.append((line[0], (fact["last"].capitalize()+" "+fact["first"]+"."+fact["middle"]+".")))
                except KeyError:
                    if fio.findall(line[1]): 
                        meta_names.append((line[0], fio.findall(line[1])[0]))
            elif fio.findall(line[1]): 
                meta_names.append((line[0], fio.findall(line[1])[0]))
        else:
            if matches:
                try:
                    for match in matches:
                        fact = match.fact.as_json 
                        meta_names.append((line[0], (fact["last"].capitalize()+" "+fact["first"]+"."+fact["middle"]+".")))
                except KeyError:
                    if fio.findall(line[1]): 
                        meta_names.append((line[0], fio.findall(line[1])[0]))
            elif fio.findall(line[1]): 
                meta_names.append((line[0], fio.findall(line[1])[0]))
    return meta_names

Функция, которая:
- приводит список с ролями и именами к упорядоченному списку без повторений в нём
- достаёт готовые метаданные с помощью parse_xml в формате словаря
- объединяет те записи из списка, для которых первое значение (роль) совпадают: (адвокат, "Петров А.П.") и (адвокат, "Семёнова Д.В.") приводятся к форме (адвокат, "Петров А.П., Семенова Д.В.")
- добавляет полученные записи из списка в словарь метаданных

In [10]:
def names_to_meta(path):
    meta_data = parse_xml(path)[0]
    updated_names = []
    unique_names = list(set(get_meta_names(meta_lines(path))))
    unique_names.sort()
    #объединение кортежей с одинаковыми нулевыми значениями 
    for i, name in enumerate(unique_names):
        if i+1 < len(unique_names):
            if name[0] == unique_names[i+1][0]:
                updated_names.append((name[0], name[1]+", "+unique_names[i+1][1]))
            elif name[0] != unique_names[i-1][0] or i == 0: 
                updated_names.append((name[0], name[1]))
        elif i+1 >= len(unique_names) and name[0] != unique_names[i-1][0]:
            updated_names.append((name[0], name[1]))
        elif len(unique_names) == 1:
            updated_names.append((name[0], name[1]))

    for name in updated_names:
        meta_data[name[0]] = name[1] 
    return meta_data

Убираем разные падежи для одного имени:

In [13]:
def killing_doubles(path):
    names_dict = names_to_meta(path)
    no_doubles_dict = {}
    for role in names_dict:
        if type(names_dict[role]) == str:
            names_list = names_dict[role].split(",") 
            if "," in names_dict[role]:
                for n, name in enumerate(names_list): 
                    if fio.findall(name): 
                        if n > 0 and name.split()[1] == names_list[n-1].split()[1]  and name.split()[0][:len(name.split()[0])-2] == names_list[n-1].split()[0][0:len(name.split()[0])-2]:
                            names_list.pop(n)
            no_doubles_dict[role] = names_list
        else:
            no_doubles_dict[role] = [names_dict[role]]
    return no_doubles_dict

Финальные метаданные:

In [148]:
def final_meta(path):
    dict_with_lists = killing_doubles(path)
    final_meta_dict = {}
    for role in dict_with_lists:
        if len(dict_with_lists[role]) == 1:
            final_meta_dict[role] = dict_with_lists[role][0]
        else:
            final_meta_dict[role] = ",".join(dict_with_lists[role])
    return final_meta_dict

Функция, которая достаёт ссылки, которые лежат внутри папки (для папки без вложенных папок):

In [105]:
def links_for_folder(path_to_folder):
    paths = []
    for f in os.walk(path_to_folder):
        for path in f[2]:
            paths.append(path_to_folder+path)
    return paths

Функция, которая возвращает словари метаданных для всех документов в папке и также возвращает количество ошибок во время извлечения:

In [106]:
def dicts_for_folder(path_to_folder):
    kerror_counter = 0
    kerror_cases = []
    terror_counter = 0
    terror_cases = []
    folder_dicts = []
    for path in links_for_folder(path_to_folder): 
        try:
            folder_dicts.append(final_meta(path))
        except KeyError: 
            kerror_counter += 1
            kerror_cases.append(path)
            continue
        except TypeError: 
            terror_counter += 1
            terror_cases.append(path)
            continue
    return folder_dicts, (kerror_counter, kerror_cases), (terror_counter, terror_cases)

Функция, которая создаёт для заданной папки датафрейм с метаинформацией:

In [107]:
def df_for_folder(path_to_folder):
    case_list = dicts_for_folder(path_to_folder)[0]
    base_df = pd.DataFrame.from_dict(case_list[0], orient = "index")
    folder_df = base_df.T
    for case in case_list[1:]:
        folder_df.loc[len(folder_df)] = pd.DataFrame.from_dict(case, orient = "index").T.loc[0]   
    return folder_df

In [114]:
dicts_for_folder("/home/laidhimonthegreen/Документы/coursework/regexp/belinskij/")

([{'number': '105539785.xml',
   'court': 'Белинский районный суд (Пензенская область)',
   'judge': 'Саунин Николай Владимирович',
   'prosecutor': 'Касаткина Н.В.',
   'advocate': 'Карцева Л.В.',
   'secretary': 'Любимкиной Т.В.',
   'accused': 'Матвеев С.И.',
   'result': 'Обвинительный приговор',
   'category': '222 ч.1'},
  {'number': '106623286.xml',
   'court': 'Белинский районный суд (Пензенская область)',
   'judge': 'Круглякова Букина Людмила Васильевна',
   'prosecutor': 'Касаткина Н.В.',
   'advocate': 'Кердяшов С.М.',
   'secretary': 'Горшковой М.А.',
   'accused': 'Грязнов А.В.',
   'result': 'Обвинительный приговор',
   'category': '158 ч.2 п.б'},
  {'number': '104670045.xml',
   'court': 'Белинский районный суд (Пензенская область)',
   'judge': 'Круглякова Букина Людмила Васильевна',
   'prosecutor': 'Касаткина Н.В.',
   'advocate': 'Кердяшов С.М.',
   'secretary': 'Шебурова В.И.',
   'accused': 'Артемов А.И.',
   'result': 'Обвинительный приговор',
   'category': '264

In [16]:
df_for_folder("/home/laidhimonthegreen/Загрузки/Telegram Desktop/zz/2012/")

,number,court,judge,prosecutor,advocate,secretary,accused,result,category
0,106227651.xml,Злынковский районный суд (Брянская область),Башлак Ирина Викторовна,Киренцов В.В.,Борисенко Н.Г.,Ница К.С.,"Бухтеев С.А., Бухтеева С.А.",Обвинительный приговор,158 ч.2; 158 ч.2 п.б; 158 ч.2 п.в
1,400192282.xml,Злынковский районный суд (Брянская область),Королёва Т. Г.,Данилова А.Ю.,Борисенко Н.Г.,Худякова М.П.,"Сироткин А.П., Сироткина А.П.",Вынесен ПРИГОВОР,162 ч.3
2,104812590.xml,Злынковский районный суд (Брянская область),Королёва Т. Г.,Лачинова Э.М.,Шишов А.В.,Худякова М.П.,"Андрюшин А.А., Андрюшина А.А.",Обвинительный приговор,158 ч.3 п.а; 158 ч.3 п.а; 158 ч.3 п.а
3,400192284.xml,Злынковский районный суд (Брянская область),Кузгумбаева Антонина Темербулатовна,Данилова А.Ю.,Борисенко Н.Г.,Поддубная Е.Н.,"Матреничев Д.А., Матреничева Д.А.",Вынесен ПРИГОВОР,158 ч.2 п.в
4,106044717.xml,Злынковский районный суд (Брянская область),Башлак Ирина Викторовна,Лачинова Э.М.,Борисенко Н.Г.,Ница К.С.,Борисенко Н.Г.,Обвинительный приговор,322 ч.1
5,104417459.xml,Злынковский районный суд (Брянская область),Кузгумбаева Антонина Темербулатовна,Данилова А.Ю.,Шишов А.В.,Поддубная Е.Н.,Коноваленко Р.А.,Обвинительный приговор,264 ч.2
6,104417457.xml,Злынковский районный суд (Брянская область),Королёва Т. Г.,Данилова А.Ю.,Борисенко Н.Г.,Поддубная Е.Н.,Пручковского А.В.,Обвинительный приговор,228 ч.2
7,102005572.xml,Злынковский районный суд (Брянская область),Кузгумбаева Антонина Темербулатовна,Данилова А.Ю.,Борисенко Н.Г.,Поддубная Е.Н.,"Кормильцев Р.П., Кормильцева Р.П.",Обвинительный приговор,158 ч.2 п.б
8,105094391.xml,Злынковский районный суд (Брянская область),Кузгумбаева Антонина Темербулатовна,Данилова А.Ю.,Борисенко Н.Г.,Поддубная Е.Н.,Борисенко Н.Г.,Обвинительный приговор,222 ч.1
9,104417456.xml,Злынковский районный суд (Брянская область),Башлак Ирина Викторовна,Данилова А.Ю.,Борисенко Н.Г.,Ница К.С.,Комовского А.Л.,Обвинительный приговор,158 ч.2 п.б


In [151]:
df_for_folder("/home/laidhimonthegreen/Загрузки/Telegram Desktop/zz/2012/")

,number,court,judge,prosecutor,advocate,secretary,accused,result,category
0,106227651.xml,Злынковский районный суд (Брянская область),Башлак Ирина Викторовна,Киренцов В.В.,Борисенко Н.Г.,Ница К.С.,Бухтеев С.А.,Обвинительный приговор,158 ч.2; 158 ч.2 п.б; 158 ч.2 п.в
1,400192282.xml,Злынковский районный суд (Брянская область),Королёва Т. Г.,Данилова А.Ю.,Борисенко Н.Г.,Худякова М.П.,Сироткин А.П.,Вынесен ПРИГОВОР,162 ч.3
2,104812590.xml,Злынковский районный суд (Брянская область),Королёва Т. Г.,Лачинова Э.М.,Шишов А.В.,Худякова М.П.,Андрюшин А.А.,Обвинительный приговор,158 ч.3 п.а; 158 ч.3 п.а; 158 ч.3 п.а
3,400192284.xml,Злынковский районный суд (Брянская область),Кузгумбаева Антонина Темербулатовна,Данилова А.Ю.,Борисенко Н.Г.,Поддубная Е.Н.,Матреничев Д.А.,Вынесен ПРИГОВОР,158 ч.2 п.в
4,106044717.xml,Злынковский районный суд (Брянская область),Башлак Ирина Викторовна,Лачинова Э.М.,Борисенко Н.Г.,Ница К.С.,Борисенко Н.Г.,Обвинительный приговор,322 ч.1
5,104417459.xml,Злынковский районный суд (Брянская область),Кузгумбаева Антонина Темербулатовна,Данилова А.Ю.,Шишов А.В.,Поддубная Е.Н.,Коноваленко Р.А.,Обвинительный приговор,264 ч.2
6,104417457.xml,Злынковский районный суд (Брянская область),Королёва Т. Г.,Данилова А.Ю.,Борисенко Н.Г.,Поддубная Е.Н.,Пручковского А.В.,Обвинительный приговор,228 ч.2
7,102005572.xml,Злынковский районный суд (Брянская область),Кузгумбаева Антонина Темербулатовна,Данилова А.Ю.,Борисенко Н.Г.,Поддубная Е.Н.,Кормильцев Р.П.,Обвинительный приговор,158 ч.2 п.б
8,105094391.xml,Злынковский районный суд (Брянская область),Кузгумбаева Антонина Темербулатовна,Данилова А.Ю.,Борисенко Н.Г.,Поддубная Е.Н.,Борисенко Н.Г.,Обвинительный приговор,222 ч.1
9,104417456.xml,Злынковский районный суд (Брянская область),Башлак Ирина Викторовна,Данилова А.Ю.,Борисенко Н.Г.,Ница К.С.,Комовского А.Л.,Обвинительный приговор,158 ч.2 п.б
